# Программирование для всех<br>(основы работы с Python)


*Алла Тамбовцева*

## Практикум 5. Работа с API ВКонтакте: собираем и обрабатываем посты со стены сообщества

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [ ]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.
По [инструкции](https://allatambov.github.io/pypolit/vk-auth.pdf) мы можем получить доступ к API, создадим приложение и скопируем его ID, чтобы получить ссылку для токена доступа:

In [ ]:
app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Переходим по ссылке выше и копируем ссылку (полностью!) из адресной строки:

In [ ]:
full_link = input()

Разбиваем полученную ссылку (сначала по `access_token=`, потом часть после `=` по `&`) и забираем токен в чистом виде:

In [ ]:
token = full_link.split("access_token=")[1].split("&")[0]

Ура! Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1. Выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [Цитатник ВШЭ](https://vk.com/hseteachers). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [ ]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "hseteachers"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [ ]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [ ]:
req_wall = requests.get(main_wall, params = params_wall)

In [ ]:
json_wall = req_wall.json()
#json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [ ]:
nposts = json_wall['response']['count']
print(nposts)

Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [ ]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [ ]:
i = items_wall[3]
i

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [ ]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [ ]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    ### YOUR CODE HERE ###
    
    time.sleep(1)
    print(i)

Проверяем длину списка – все ли посты собраны:

In [ ]:
len(items_all)

### Задача 4

Создайте на основе списка `items_all` датафрейм `df` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

In [ ]:
import pandas as pd

In [ ]:
### YOUR CODE HERE ###

## Часть 2: обрабатываем посты ВКонтакте

В файле `hseteachers.xlsx` сохранены посты со стены сообщества Цитатник ВШЭ:

* `id`: id поста;
* `date`: дата-время публикации поста в формате POSIX;
* `text`: текст поста;
* `nlikes`: число лайков;
* `ncomments`: число комментариев. 

Загрузите данные из файла и сохраните их в датафрейм Pandas. 

In [ ]:
### YOUR CODE HERE ###

### Задача 1

Разбейте столбец `text` по символу `#`, чтобы получить три столбца:

* `words`: текст цитаты;
* `teacher`: имя преподавателя (как в тексте поста);
* `subject`: название курса/программы/факультета, где работает преподаватель (как в тексте поста).

Сам столбец `text` не удаляйте, просто добавьте новые три столбца к имеющемуся датафрейму. 

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Найдите преподавателя, чей пост набрал больше всего лайков, двумя способами:

* отсортировав строки по количеству лайков по убыванию;
* зафиксировав максимум и отфильтровав строки.

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Напишите программу, которая запрашивает с клавиатуры названия трёх курсов/программ через запятую и сохраняет в файл Excel строки датафрейма, соответствующие запросу.

In [ ]:
### YOUR CODE HERE ###

### Задача 4

Добавьте в датафрейм столбец `date_time` с датой и временем поста в формате *datetime*. Хранить метку времени в таком формате удобно, так как, во-первых, дата и время представлены в понятном формате, а во-вторых, при сортировке или построении графиков для динамики строки будут упорядочиваться в правильном хронологическом порядке.

**Подсказка:** функция `.to_datetime()`.

In [ ]:
### YOUR CODE HERE ###

### Задача 5

Проделайте аналогичную операцию с метками времени, только представьте их не в формате *datetime*, а в виде обычных строк (тип *object*, который в Pandas соответствует базовому типу *string*). Используя форматирование строк для дат, извлеките название месяца и название дня недели и сохраните их в столбцы `month` и `weekday` соответственно.

In [ ]:
### YOUR CODE HERE ###

### Задача 6

Сгруппируйте строки по дням недели и определите, в какие дни недели:

* публикуется больше всего/меньше всего постов;
* посты получают наибольшее/наименьшее число комментариев.

In [ ]:
### YOUR CODE HERE ###

### Задача 7

Выберите строки датафрейма, который соответствуют первому модулю 2023-2024 учебного года и постройте для полученного датафрейма график, отражающий динамику числа лайков в течение модуля (по горизонтальной оси дата, по вертикальной – суммарное число лайков в день).

In [ ]:
### YOUR CODE HERE ###

### Задача 8

Выберите строки датафрейма, соответствующие постам, число комментариев к которым больше 0. Переименуйте столбец `id` в `post_id`. 

In [ ]:
### YOUR CODE HERE ###

### Задача 9*

Используя API ВКонтакте, выгрузите комментарии к каждому посту в датафрейме, полученном в предыдущей задаче, и сохраните их в виде датафрейма. В датафрейме с комментариями должна остаться следующая информация: 

* id комментария;
* дата комментария;
* текст комментария;
* id пользователя, который оставил комментарий.

Объедините датафреймы с постами и с комментариями по id поста таким образом, чтобы в итоговом датафрейме одна строка соответствовала одному комментарию (комментарии уникальны, посты не уникальны, так как одному посту может соответствовать несколько комментариев).

In [ ]:
### YOUR CODE HERE ###

### Задача 10*

Используя API ВКонтакте, выгрузите следующую информацию по пользователям, оставившим комментарии к постам, то есть по тем пользователям, чей id зафиксирован в датафрейме, полученном в предыдущей задаче:
 
* имя пользователя;
* вуз (если указан).

«Подтяните» полученную информацию к датафрейму с комментариями, то есть добавьте туда столбцы с именем пользователя и вузом. 

In [ ]:
### YOUR CODE HERE ###